In [1]:
#
# Se crea el directorio de entrada
#
!rm -rf /tmp/wordcount
!mkdir -p /tmp/wordcount/input
%cd /tmp/wordcount
!ls

/tmp/wordcount
input


In [2]:
%%writefile input/text0.txt
Analytics is the discovery, interpretation, and communication of meaningful patterns
in data. Especially valuable in areas rich with recorded information, analytics relies
on the simultaneous application of statistics, computer programming and operations research
to quantify performance.

Organizations may apply analytics to business data to describe, predict, and improve business
performance. Specifically, areas within analytics include predictive analytics, prescriptive
analytics, enterprise decision management, descriptive analytics, cognitive analytics, Big
Data Analytics, retail analytics, store assortment and stock-keeping unit optimization,
marketing optimization and marketing mix modeling, web analytics, call analytics, speech
analytics, sales force sizing and optimization, price and promotion modeling, predictive
science, credit risk analysis, and fraud analytics. Since analytics can require extensive
computation (see big data), the algorithms and software used for analytics harness the most
current methods in computer science, statistics, and mathematics.

Writing input/text0.txt


In [3]:
%%writefile input/text1.txt
The field of data analysis. Analytics often involves studying past historical data to
research potential trends, to analyze the effects of certain decisions or events, or to
evaluate the performance of a given tool or scenario. The goal of analytics is to improve
the business by gaining knowledge which can be used to make improvements or changes.

Writing input/text1.txt


In [4]:
%%writefile input/text2.txt
Data analytics (DA) is the process of examining data sets in order to draw conclusions
about the information they contain, increasingly with the aid of specialized systems
and software. Data analytics technologies and techniques are widely used in commercial
industries to enable organizations to make more-informed business decisions and by
scientists and researchers to verify or disprove scientific models, theories and
hypotheses.

Writing input/text2.txt


In [5]:
%%writefile word_counter.py

import string

from mrjob.job import MRJob


class WordCounter(MRJob):
    def preprocessing(self, word):
        word = word.lower()
        word = word.translate(str.maketrans("", "", string.punctuation))
        word = word.replace("\n", "")
        return word

    def mapper(self, _, line):
        for word in line.split():
            word = self.preprocessing(word)
            yield word, 1

    def reducer(self, key, values):
        yield key, sum(values)


if __name__ == "__main__":
    WordCounter.run()

Writing word_counter.py


In [6]:
!python3 word_counter.py  input/*.txt | head

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/word_counter.root.20230317.110111.475777
Running step 1 of 1...
job output is in /tmp/word_counter.root.20230317.110111.475777/output
Streaming final output from /tmp/word_counter.root.20230317.110111.475777/output...
Removing temp directory /tmp/word_counter.root.20230317.110111.475777...
"a"	1
"about"	1
"aid"	1
"algorithms"	1
"analysis"	2
"analytics"	20
"analyze"	1
"and"	15
"application"	1
"apply"	1


In [7]:
!hadoop fs -mkdir input
!hadoop fs -copyFromLocal input/* input
!hadoop fs -ls input/*

-rw-r--r--   1 root supergroup       1082 2023-03-17 11:01 input/text0.txt
-rw-r--r--   1 root supergroup        349 2023-03-17 11:01 input/text1.txt
-rw-r--r--   1 root supergroup        435 2023-03-17 11:01 input/text2.txt


In [8]:
!python3 word_counter.py -r hadoop hdfs:///user/root/input | head -n 20

No configs found; falling back on auto-configuration
No configs specified for hadoop runner
Looking for hadoop binary in /opt/hadoop/bin...
Found hadoop binary: /opt/hadoop/bin/hadoop
Using Hadoop version 2.10.1
Looking for Hadoop streaming jar in /opt/hadoop...
Found Hadoop streaming jar: /opt/hadoop/share/hadoop/tools/lib/hadoop-streaming-2.10.1.jar
Creating temp directory /tmp/word_counter.root.20230317.110117.509864
uploading working dir files to hdfs:///user/root/tmp/mrjob/word_counter.root.20230317.110117.509864/files/wd...
Copying other local files to hdfs:///user/root/tmp/mrjob/word_counter.root.20230317.110117.509864/files/
Running step 1 of 1...
  packageJobJar: [/tmp/hadoop-unjar2968620037757923210/] [] /tmp/streamjob1830253130673648290.jar tmpDir=null
Attempting to fetch counters from logs...
Can't fetch history log; missing job ID
No counters found
job output is in hdfs:///user/root/tmp/mrjob/word_counter.root.20230317.110117.509864/output
Streaming final output from hdfs:

In [9]:
!python3 word_counter.py -r hadoop hdfs:///user/root/input -o hdfs:///user/root/output

No configs found; falling back on auto-configuration
No configs specified for hadoop runner
Looking for hadoop binary in /opt/hadoop/bin...
Found hadoop binary: /opt/hadoop/bin/hadoop
Using Hadoop version 2.10.1
Looking for Hadoop streaming jar in /opt/hadoop...
Found Hadoop streaming jar: /opt/hadoop/share/hadoop/tools/lib/hadoop-streaming-2.10.1.jar
Creating temp directory /tmp/word_counter.root.20230317.110200.911176
uploading working dir files to hdfs:///user/root/tmp/mrjob/word_counter.root.20230317.110200.911176/files/wd...
Copying other local files to hdfs:///user/root/tmp/mrjob/word_counter.root.20230317.110200.911176/files/
Running step 1 of 1...
  packageJobJar: [/tmp/hadoop-unjar894766942577585846/] [] /tmp/streamjob5180413273604877256.jar tmpDir=null
Attempting to fetch counters from logs...
Can't fetch history log; missing job ID
No counters found
job output is in hdfs:///user/root/output
Removing HDFS temp directory hdfs:///user/root/tmp/mrjob/word_counter.root.20230317.1

In [10]:
!hadoop fs -ls output

Found 2 items
-rw-r--r--   1 root supergroup          0 2023-03-17 11:02 output/_SUCCESS
-rw-r--r--   1 root supergroup       1742 2023-03-17 11:02 output/part-00000


In [11]:
!hadoop fs -cat output/part-00000 | head -n 10

"a"	1
"about"	1
"aid"	1
"algorithms"	1
"analysis"	2
"analytics"	20
"analyze"	1
"and"	15
"application"	1
"apply"	1


In [12]:
!hadoop fs -rm -r output
!hadoop fs -rm -r tmp
!hadoop fs -rm -r input

Deleted output
Deleted tmp
Deleted input
